# Section 1
You would have received a Access_data.zip file with 22 datasets in .csv format and 1 data dictionary in .docx format. The datasets are sample of the mock-data on individuals accessing two office sites of Company ABC, consisting of:

1. When: Time of entry by the individual

2. Profile: Type of access card
    -	0 - Staff Pass
    -	1 - Temp Pass
    -	2 - Visitor Pass

3. Dept: Department of the individual

4. CardNum: Card unique identifier. The length of the card number cannot be less than 8 characters. Currently, if CardNum starts with a/multiple ‘0’, the data captured in system will exclude/remove the “0”.

You can assume that the **total number of staff in the company is 2000 and the data is extracted from the company’s building access system.** An individual can tap in and out several times within the same day. When the individual first clock in, that would be the earliest time slot and the only record you will base off the analysis. (You can also state your other assumptions if need be.)


In [1]:
## Imports
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import statsmodels as sm
import os

## Question 1.1
Write code preferably in R or Python to process and **organise the raw data (Access_data.zip) to make it suitable for analysis**. Identify and resolve the data quality issues in the raw data, if any. (The created code should allow user to efficiently and easily run it to ingest additional datasets of different period, beyond the given sample.)

We assume the files are represented in the format of 'SiteAYYYYMMDD-YYYYMMDDa.csv' or 'SiteBYYYYMMDD-YYYYMMDDb.csv' for both sites respectively.

In [2]:
# Utility function for listing particular required site files in a specified data directory path
def list_files_for_a_site(site_name):
    data_dir_path=os.path.join(os.getcwd(), "data", "Access_Data")
    return [os.path.join(data_dir_path, file) for file in os.listdir(data_dir_path) if file.startswith(site_name)]

In [3]:
# Construct list of site A and site B files.
site_A_file_list =  list_files_for_a_site(site_name="SiteA")
site_B_file_list =  list_files_for_a_site(site_name="SiteB")

print(site_A_file_list)
print(site_B_file_list)

['c:\\Users\\Admin\\Desktop\\MOM_Senior-Analyst-Analyst_SensingAnalytics_Assessment\\data\\Access_Data\\SiteA20200420-20200426a.csv', 'c:\\Users\\Admin\\Desktop\\MOM_Senior-Analyst-Analyst_SensingAnalytics_Assessment\\data\\Access_Data\\SiteA20200427-20200503a.csv', 'c:\\Users\\Admin\\Desktop\\MOM_Senior-Analyst-Analyst_SensingAnalytics_Assessment\\data\\Access_Data\\SiteA20200504-20200510a.csv', 'c:\\Users\\Admin\\Desktop\\MOM_Senior-Analyst-Analyst_SensingAnalytics_Assessment\\data\\Access_Data\\SiteA20200511-20200517a.csv', 'c:\\Users\\Admin\\Desktop\\MOM_Senior-Analyst-Analyst_SensingAnalytics_Assessment\\data\\Access_Data\\SiteA20200518-20200524a.csv', 'c:\\Users\\Admin\\Desktop\\MOM_Senior-Analyst-Analyst_SensingAnalytics_Assessment\\data\\Access_Data\\SiteA20200525-20200531a.csv', 'c:\\Users\\Admin\\Desktop\\MOM_Senior-Analyst-Analyst_SensingAnalytics_Assessment\\data\\Access_Data\\SiteA20200601a.csv', 'c:\\Users\\Admin\\Desktop\\MOM_Senior-Analyst-Analyst_SensingAnalytics_Asses

In [4]:
def column_checker(df, filename):
    expected_col = set(["When", "Profile", "Dept", "CardNum"])
    symmetric_diff_set = set(df.columns).symmetric_difference(expected_col)
    if symmetric_diff_set :
        print(f"Identified a non-expected column for {filename}")
        print(f"Symmetric difference: {symmetric_diff_set}")
    return None

### Process site A and site B

Do a quick check on column name and found that one of site A data file has column named "Depts" instead of "Dept" while for site B, there is a column named "CardNum " instead of "CardNum" for same period of 20200622-20200628. To resolve this issue, we will strip all leading/trailing space and extract the first 4 alphanumeric representation for convenience

In [5]:
for file in site_A_file_list:
    temp_df = pd.read_csv(file, sep=",")
    column_checker(temp_df, file)

print()
for file in site_B_file_list:
    temp_df = pd.read_csv(file, sep=",")
    column_checker(temp_df, file)

Identified a non-expected column for c:\Users\Admin\Desktop\MOM_Senior-Analyst-Analyst_SensingAnalytics_Assessment\data\Access_Data\SiteA20200622-20200628a.csv
Symmetric difference: {'Depts', 'Dept'}

Identified a non-expected column for c:\Users\Admin\Desktop\MOM_Senior-Analyst-Analyst_SensingAnalytics_Assessment\data\Access_Data\SiteB20200622-20200628b.csv
Symmetric difference: {'CardNum', 'CardNum '}


## Data loading
Load the each site's access data files as a dataframe into a list for vertical stacking. We assume the time period for both sites will be the period between Apr 20, 2020 to Jun 28, 2020 as per the on file name date representation.

Notice that there are quite a significant number of nulls for Department feature in Site A (9266) and Site B (10100); and 5 null card information for site B.

In [6]:
site_A_df_list = []
for file in site_A_file_list:
    temp_df = pd.read_csv(file, sep=",")
    temp_df.columns = [col.strip()[:4] for col in temp_df.columns]
    site_A_df_list.append(temp_df)

site_B_df_list = []
for file in site_B_file_list:
    temp_df = pd.read_csv(file, sep=",")
    temp_df.columns = [col.strip()[:4] for col in temp_df.columns]
    site_B_df_list.append(temp_df)


site_A_df = pd.concat(site_A_df_list, ignore_index=True)
site_B_df = pd.concat(site_B_df_list, ignore_index=True)
print(site_A_df.info())
print(site_B_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12192 entries, 0 to 12191
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   When    12192 non-null  object
 1   Prof    12192 non-null  object
 2   Dept    2925 non-null   object
 3   Card    12192 non-null  object
dtypes: object(4)
memory usage: 381.1+ KB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24499 entries, 0 to 24498
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   When    24499 non-null  object
 1   Prof    24499 non-null  object
 2   Dept    14399 non-null  object
 3   Card    24494 non-null  object
dtypes: object(4)
memory usage: 765.7+ KB
None


## Managing Department and Profile features - Site A
Check Department and Profile uniqueness for both sides as they are categorical. Notice that for Site A, we see that there is some form of discrepancy for Dept 1, as well as the Profile value represented in numeric or string format.

To simplify department representation, we will remove all spaces and concatenate the alphanumeric representation, while for profile representation, we will do a string cast and standardise to meaningful name representation

In [7]:
# Convert card type to string in case of int and string mix representation
site_A_df["Card"] = site_A_df["Card"].astype(str)
print(site_A_df["Dept"].unique())
print(site_A_df["Prof"].unique())

[nan 'Dept 5' 'Dept 11' 'Dept 18' 'Dept 4' 'Dept 9' 'Dept 15' 'Dept 14'
 'Dept 2' 'Dept 8' 'Dept 12' 'Dept  1' 'Dept 1' 'Dept 19' 'Dept 7'
 'Dept 17' 'Dept 10' 'Dept 6' 'Dept 3' 'Dept 16' 'Dept 13']
[2 1 0 '1' '0' '2' 'Visitor Pass']


In [8]:
profile_pass_mapping = {
    "0": "Staff Pass",
    "1": "Temp Pass",
    "2": "Visitor Pass"
}

site_A_df["Dept"] = site_A_df["Dept"].fillna("unknown")
site_A_df["Dept"] = site_A_df["Dept"].map(lambda x: x.replace(" ","") if x else x)

site_A_df["Prof"] = site_A_df["Prof"].map(lambda x: str(x))
site_A_df["Prof"] = site_A_df["Prof"].map(lambda x: profile_pass_mapping[x] if x in profile_pass_mapping else x)

In [9]:
print(site_A_df["Dept"].unique())
print(site_A_df["Prof"].unique())

['unknown' 'Dept5' 'Dept11' 'Dept18' 'Dept4' 'Dept9' 'Dept15' 'Dept14'
 'Dept2' 'Dept8' 'Dept12' 'Dept1' 'Dept19' 'Dept7' 'Dept17' 'Dept10'
 'Dept6' 'Dept3' 'Dept16' 'Dept13']
['Visitor Pass' 'Temp Pass' 'Staff Pass']


## Null data handling: Site A
Handle missing data for Site A involving department feature. Identify the pass type (profile) which the department information is unknown. We need to check from the perspective of each pass type(profile) and see the applicability of department before deciding how to impute.

Checking from this 2 perspective, we conclude that the department information is not applicable for both temp/visitor pass. We can fill as not applicable representation.

In [10]:
# Identify the pass type (profile) which the department information is unknown, as we need to impute
site_A_df[site_A_df["Dept"]=="unknown"]["Prof"].unique()

array(['Visitor Pass', 'Temp Pass'], dtype=object)

In [11]:
# For each pass type (profile), see its applicability to department information
for profile in site_A_df["Prof"].unique():
    unique_dept = site_A_df[site_A_df["Prof"]==profile]["Dept"].unique()
    print(f"{profile}:{unique_dept}")

Visitor Pass:['unknown']
Temp Pass:['unknown']
Staff Pass:['Dept5' 'Dept11' 'Dept18' 'Dept4' 'Dept9' 'Dept15' 'Dept14' 'Dept2'
 'Dept8' 'Dept12' 'Dept1' 'Dept19' 'Dept7' 'Dept17' 'Dept10' 'Dept6'
 'Dept3' 'Dept16' 'Dept13']


## Managing Department and Profile features - Site B
Check Department and Profile uniqueness for both sides as they are categorical. Notice that for Site B, we see that the Profile value represented in numeric or string format.

To simplify department representation, we will use the same approach for this site remove all spaces and concatenate the alphanumeric representation, while for profile representation, we will do a string cast and standardise to meaningful name representation.

In [12]:
# Convert card type to string in case of int and string mix representation
site_B_df["Card"] = site_B_df["Card"].astype(str)
print(site_B_df["Dept"].unique())
print(site_B_df["Prof"].unique())

['Dept 4' nan 'Dept 2' 'Dept 16' 'Dept 10' 'Dept 5' 'Dept 14' 'Dept 3'
 'Dept 15' 'Dept 8' 'Dept 13' 'Dept 19' 'Dept 11' 'Dept 9' 'Dept 18'
 'Dept 7' 'Dept 17' 'Dept 1' 'Dept 12' 'Dept 6']
[0 1 2 '0' '1' '2' 'Temp Pass' 'Staff Pass']


Convert card value as string results in null value resolved.

In [13]:
site_B_df["Card"].notnull().all()

True

In [14]:
site_B_df["Dept"] = site_B_df["Dept"].fillna("unknown")
site_B_df["Dept"] = site_B_df["Dept"].map(lambda x: x.replace(" ","") if x else x)

site_B_df["Prof"] = site_B_df["Prof"].map(lambda x: str(x))
site_B_df["Prof"] = site_B_df["Prof"].map(lambda x: profile_pass_mapping[x] if x in profile_pass_mapping else x)

In [15]:
print(site_B_df["Dept"].unique())
print(site_B_df["Prof"].unique())

['Dept4' 'unknown' 'Dept2' 'Dept16' 'Dept10' 'Dept5' 'Dept14' 'Dept3'
 'Dept15' 'Dept8' 'Dept13' 'Dept19' 'Dept11' 'Dept9' 'Dept18' 'Dept7'
 'Dept17' 'Dept1' 'Dept12' 'Dept6']
['Staff Pass' 'Temp Pass' 'Visitor Pass']


Handle missing data for Site B involving Department feature.

For Department, identify the profile which department info is unknown. We also need to check from the perspective of each pass type(profile) and see the uniqueness of department to decide how to impute.

Checking from this 2 perspective, we conclude that the department information is not applicable for both temp/visitor pass. We can fill as not applicable representation.

In [16]:
site_B_df[site_B_df["Dept"]=="unknown"]["Prof"].unique()

array(['Temp Pass', 'Visitor Pass'], dtype=object)

In [17]:
for profile in site_B_df["Prof"].unique():
    unique_dept = site_B_df[site_B_df["Prof"]==profile]["Dept"].unique()
    print(f"{profile}:{unique_dept}")

Staff Pass:['Dept4' 'Dept2' 'Dept16' 'Dept10' 'Dept5' 'Dept14' 'Dept3' 'Dept15'
 'Dept8' 'Dept13' 'Dept19' 'Dept11' 'Dept9' 'Dept18' 'Dept7' 'Dept17'
 'Dept1' 'Dept12' 'Dept6']
Temp Pass:['unknown']
Visitor Pass:['unknown']


In both sites, we are sure that temp and visitor pass do not have corresponding department info. As such we will replace it with 'not_applicable' value instead of 'unknown' as a meaningful representation.

In [18]:
site_A_df["Dept"] = site_A_df["Dept"].map(lambda x:x.replace("unknown","not_applicable" if x=="unknown" else x))
site_B_df["Dept"] = site_B_df["Dept"].map(lambda x:x.replace("unknown","not_applicable" if x=="unknown" else x))

In [19]:
site_A_df.columns

Index(['When', 'Prof', 'Dept', 'Card'], dtype='object')

## Managing card feature 
We still have null cases where card information is unknown. A quick check shows the card affected are either temp pass/visitor pass. We may want to combine the dataframe from 2 sites to resolve the card information issue as no additional information are provided.

In [20]:
print(site_A_df.shape, site_B_df.shape)

(12192, 4) (24499, 4)


In [37]:
# Construct useful identifier features
site_B_df["Site"] = "B"
site_B_df["Dept"] = site_B_df["Dept"].map(lambda x: x+"_B")
site_A_df["Site"] = "A"
site_A_df["Dept"] = site_A_df["Dept"].map(lambda x: x+"_A")

In [38]:
# Stack the dataframe vertically
both_sites_df = pd.concat([site_A_df, site_B_df], ignore_index=True)
both_sites_df.head()

,When,Prof,Dept,Card,Site
0,20/4/2020 7:17,Visitor Pass,not_applicable_A,1001,A
1,21/4/2020 7:10,Visitor Pass,not_applicable_A,1001,A
2,22/4/2020 7:09,Visitor Pass,not_applicable_A,1001,A
3,23/4/2020 7:16,Visitor Pass,not_applicable_A,1001,A
4,24/4/2020 7:25,Visitor Pass,not_applicable_A,1001,A


In [39]:
both_sites_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36691 entries, 0 to 36690
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   When    36691 non-null  object
 1   Prof    36691 non-null  object
 2   Dept    36691 non-null  object
 3   Card    36691 non-null  object
 4   Site    36691 non-null  object
dtypes: object(5)
memory usage: 1.4+ MB


In [40]:
# Check formatting of When feature
pd.to_datetime(both_sites_df["When"], format="%d/%m/%Y %H:%M", errors="coerce").notnull().all()

True

In [41]:
# Convert to datetime and extract date and time info separately
both_sites_df["When"] = pd.to_datetime(both_sites_df["When"], format="%d/%m/%Y %H:%M")
# Split When into Date and TIme
both_sites_df['Date'] = both_sites_df['When'].dt.date
both_sites_df['Time'] = both_sites_df['When'].dt.time

Construct time period categorisation based on 0700-0829, 0830 to 0929 and 0930 to 1030 categories

In [63]:
import datetime
def time_in_range(x):
    """Return true if x is in the range [start, end]"""
    if datetime.time(7,0,0) <= x <=datetime.time(8,29,0):
        return "0700-0829"
    elif datetime.time(8,30,0) <= x <=datetime.time(9,29,0):
        return "0830-0929"
    elif datetime.time(9,30,0) <= x <=datetime.time(10,30,0):
        return "0930-1030"
    elif x< datetime.time(7,0,0):
        return "Before 0700"
    else:
        return "After 1030"

In [66]:
both_sites_df["Accessperiod"] = both_sites_df["Time"].map(time_in_range)
both_sites_df["Accessperiod"].value_counts()

Accessperiod
0700-0829       18187
0830-0929       12734
0930-1030        5216
WorkFromHome      554
Name: count, dtype: int64

### Investigate card number in detail after typecasting into string as it may not be numeric.

Segregate data into numeric and non numeric dataframes based on card info.
A quick check shows 871 of such records.

In [42]:
both_sites_df["is_numeric_Card"] = both_sites_df["Card"].map(lambda x: x.isnumeric())

In [43]:
# Split dataframe into 2 sets based on card is fully numeric or not
non_numeric_card_records_df = both_sites_df[both_sites_df["is_numeric_Card"]==False]
numeric_card_records_df = both_sites_df[both_sites_df["is_numeric_Card"]==True]
print(non_numeric_card_records_df.shape, numeric_card_records_df.shape)

(871, 8) (35820, 8)


Identify if it is possible for a card length to exceed 8 by checking all department code (including non_applicable which applies to temp/visitor).

From the result, the data suggests that the max digits is 8 when checking against all departments. As such, we assume that the card number is only 8 characters.

In [45]:
dept_list = [dept for dept in both_sites_df["Dept"].unique()]

# Check card length for each staff pass. It can be either 8 or more characters.
for dept in dept_list:
    card_list = numeric_card_records_df[numeric_card_records_df["Dept"]==dept]["Card"].unique()
    max_val = max([len(x) for x in card_list])
    print(dept, max_val)
    

not_applicable_A 8
Dept5_A 7
Dept11_A 7
Dept18_A 7
Dept4_A 6
Dept9_A 7
Dept15_A 7
Dept14_A 7
Dept2_A 7
Dept8_A 7
Dept12_A 7
Dept1_A 7
Dept19_A 7
Dept7_A 7
Dept17_A 7
Dept10_A 6
Dept6_A 6
Dept3_A 7
Dept16_A 7
Dept13_A 5
Dept4_B 7
not_applicable_B 8
Dept2_B 7
Dept16_B 7
Dept10_B 7
Dept5_B 6
Dept14_B 7
Dept3_B 7
Dept15_B 7
Dept8_B 7
Dept13_B 6
Dept19_B 7
Dept11_B 6
Dept9_B 7
Dept18_B 7
Dept7_B 6
Dept17_B 7
Dept1_B 7
Dept12_B 7
Dept6_B 7


In [46]:
# Fill 0s in the front to restore original card ID
numeric_card_records_df["Card"] = numeric_card_records_df["Card"].map(lambda x: str(x).zfill(8))

C:\Users\Admin\AppData\Local\Temp\ipykernel_19324\67502454.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  numeric_card_records_df["Card"] = numeric_card_records_df["Card"].map(lambda x: str(x).zfill(8))


Check if card ID is shared by staff in both sites department. The results suggests that some cards are identified with different departments, which can mean that the staff is concurrently holding different appointments at different departments which was registered by the system when tapped at separate office OR a switch in appointment in the absence of time factor.

To simplify analysis, the affected card will be remapped to a combination of the department information instead.

In [47]:
# Consider department which are valid 
valid_dept_list = [dept for dept in both_sites_df["Dept"].unique() if dept.startswith("Dept")]

dept_card_dict = {}
for dept in valid_dept_list:
    
    card_list = set(numeric_card_records_df[numeric_card_records_df["Dept"]==dept]["Card"].unique())
    dept_card_dict[dept] = card_list

In [49]:
# Check if same card ID is mapped to differing department.
from itertools import combinations
dept_combinations = combinations(dept_card_dict, 2)
card_with_multi_dept = set()
for dept1, dept2 in dept_combinations:
    intersection = dept_card_dict[dept1].intersection(dept_card_dict[dept2])
    if intersection:
        print(f"Common card used for {dept1}, {dept2}: {intersection}")
        # Update set to identify all affected card holders
        card_with_multi_dept.update(list(intersection))
    

Common card used for Dept5_A, Dept11_A: {'00004402'}
Common card used for Dept5_A, Dept18_A: {'00008202'}
Common card used for Dept5_A, Dept15_A: {'00005103'}
Common card used for Dept5_A, Dept14_A: {'00002502', '00008704'}
Common card used for Dept5_A, Dept4_B: {'00408202', '00000902', '00771858', '00821802', '00254402', '00333803', '00295302', '00095902', '00335103', '00782409', '00438704', '00820591'}
Common card used for Dept5_A, Dept16_B: {'00004402'}
Common card used for Dept5_A, Dept5_B: {'00408202', '00771858', '00830595', '00165105', '00821802', '00254402', '00029202', '00333803', '00181802', '00295302', '00095902', '00335103', '00782409', '00341404', '00438704', '00820591', '00003802'}
Common card used for Dept5_A, Dept14_B: {'00005302'}
Common card used for Dept5_A, Dept3_B: {'00000902'}
Common card used for Dept5_A, Dept8_B: {'00003802'}
Common card used for Dept11_A, Dept18_A: {'00002902'}
Common card used for Dept11_A, Dept2_A: {'00002902'}
Common card used for Dept11_A, 

In [53]:
# Check number of staff whose card is tagged to multiple department
print(len(card_with_multi_dept))

print(numeric_card_records_df[numeric_card_records_df["Prof"]=="Staff Pass"]["Card"].nunique())

216
1516


In [55]:
for card in card_with_multi_dept:
    numeric_card_records_df.loc[numeric_card_records_df["Card"] == card, "Dept"] = "MultiDept"

Dept
not_applicable_B    9347
not_applicable_A    9170
Dept4_B             8214
MultiDept           3433
Dept16_B            1445
Dept18_A             592
Dept10_B             572
Dept2_B              525
Dept8_A              259
Dept15_A             246
Dept11_A             225
Dept9_B              222
Dept5_A              214
Dept18_B             209
Dept14_B             161
Dept19_B             147
Dept14_A             144
Dept8_B              127
Dept2_A              100
Dept17_A              79
Dept1_A               68
Dept13_B              57
Dept15_B              48
Dept3_B               43
Dept12_A              32
Dept5_B               31
Dept19_A              20
Dept10_A              16
Dept7_B               15
Dept17_B              14
Dept9_A               12
Dept11_B              11
Dept7_A                6
Dept1_B                6
Dept6_B                4
Dept12_B               3
Dept16_A               2
Dept4_A                1
Name: count, dtype: int64

Handle possible duplicates

In [62]:
staff_numeric_card_records_df = numeric_card_records_df[numeric_card_records_df["Prof"]=="Staff Pass"]
non_staff_numeric_card_records_df = numeric_card_records_df[numeric_card_records_df["Prof"]!="Staff Pass"]

print(len(staff_numeric_card_records_df),len(non_staff_numeric_card_records_df))

17248 18572


In [58]:
# Get breakdown of cards with multiple entries per date
duplicated_df = numeric_card_records_df[numeric_card_records_df.duplicated(subset=["Date","Dept","Card"], keep=False)]
duplicated_df.shape

(5371, 8)

In [60]:
# Get a sample for staff cases sorted by "Date"
duplicated_df[duplicated_df["Prof"]=="Staff Pass"].sort_values("Date")

,When,Prof,Dept,Card,Site,Date,Time,is_numeric_Card
12510,2020-04-20 09:03:00,Staff Pass,Dept4_B,00120502,B,2020-04-20,09:03:00,True
12512,2020-04-20 08:42:00,Staff Pass,Dept4_B,00120502,B,2020-04-20,08:42:00,True
14941,2020-04-28 08:57:00,Staff Pass,Dept4_B,00120502,B,2020-04-28,08:57:00,True
14945,2020-04-28 08:44:00,Staff Pass,Dept4_B,00120502,B,2020-04-28,08:44:00,True
15049,2020-04-28 08:06:00,Staff Pass,Dept4_B,00172302,B,2020-04-28,08:06:00,True
...,...,...,...,...,...,...,...,...
34496,2020-06-26 08:25:00,Staff Pass,Dept4_B,00172302,B,2020-06-26,08:25:00,True
34500,2020-06-26 08:31:00,Staff Pass,Dept4_B,00172302,B,2020-06-26,08:31:00,True
10831,2020-06-26 09:31:00,Staff Pass,MultiDept,00119102,A,2020-06-26,09:31:00,True
34497,2020-06-27 08:06:00,Staff Pass,Dept4_B,00172302,B,2020-06-27,08:06:00,True


In [ ]:
# Drop duplicates for dataframe containing known card records using date, site and card info as we only need the first instance.
print(numeric_card_records_df.shape)
numeric_card_records_df = numeric_card_records_df.drop_duplicates(subset=["Date","Site","Card"], keep="first")
numeric_card_records_df.shape

(35820, 8)


(32533, 8)

In [ ]:
# Check dates which such records are affected
non_numeric_card_records_df["Date"].unique()

array([datetime.date(2020, 4, 20), datetime.date(2020, 5, 28),
       datetime.date(2020, 5, 27), datetime.date(2020, 6, 1),
       datetime.date(2020, 4, 21), datetime.date(2020, 4, 25),
       datetime.date(2020, 4, 23), datetime.date(2020, 4, 22),
       datetime.date(2020, 4, 24), datetime.date(2020, 4, 26),
       datetime.date(2020, 4, 30), datetime.date(2020, 4, 28),
       datetime.date(2020, 5, 3), datetime.date(2020, 5, 4),
       datetime.date(2020, 5, 7), datetime.date(2020, 5, 6),
       datetime.date(2020, 5, 8), datetime.date(2020, 5, 29),
       datetime.date(2020, 5, 26), datetime.date(2020, 6, 3),
       datetime.date(2020, 6, 7), datetime.date(2020, 6, 4),
       datetime.date(2020, 6, 23)], dtype=object)

Get the breakdown of profile, department and site which card info is non numeric. It was noted that the #REF indicate some form of card number reference to other row and #VALUE represents invalid value in excel file context.

Records suggest that Site B seems to be impacted heavily with missing or invalid card information during the period of 20 to 26 Apr 2020, totaling about 600+ instances. An abnormaly is noticed for Site A on 1 Jun 2020, with 106 instances. For such cases, temp pass are largely affected as compared to staff pass based on breakdown of processed information.

### Additional assumptions

Assume that temporary/visitor passes are not sharable between sites.


In [ ]:
aaa

NameError: name 'aaa' is not defined

In [ ]:
# Get the daily records which card information is missing
non_numeric_card_records_df[["Date","Site","Prof"]].value_counts().sort_index().plot()

NameError: name 'non_numeric_card_records_df' is not defined

In [ ]:
# Get the daily records which card information is missing
non_numeric_card_records_df[["Date","Site"]].value_counts().sort_index()

Date        Site
2020-04-20  A         1
            B       113
2020-04-21  B       120
2020-04-22  B       126
2020-04-23  B       114
2020-04-24  B       123
2020-04-25  B        80
2020-04-26  B        71
2020-04-28  B         1
2020-04-30  B         1
2020-05-03  B         1
2020-05-04  B         1
2020-05-06  B         1
2020-05-07  B         1
2020-05-08  B         1
2020-05-26  B         1
2020-05-27  A         1
2020-05-28  A         2
            B         1
2020-05-29  B         1
2020-06-01  A       106
2020-06-03  B         1
2020-06-04  B         1
2020-06-07  B         1
2020-06-23  B         1
Name: count, dtype: int64

Construct a dictionary storing unique card tap-ins per day per department

In [ ]:
valid_dept_list = [dept for dept in both_sites_df["Dept"].unique() if dept.startswith("Dept")]

daily_dates_list = [date for date in both_sites_df["Date"].unique()]

staff_card_daily_info = {k:{} for k in valid_dept_list}


for dept in valid_dept_list:
    for date in daily_dates_list:
        unique_card_list = both_sites_df[(both_sites_df["Date"]==date) & (both_sites_df["Dept"]==dept)]["Card"].unique()
        staff_card_daily_info[dept][date] = len(unique_card_list)

In [ ]:
staff_card_daily_info

{'Dept5': {datetime.date(2020, 4, 20): 9,
  datetime.date(2020, 4, 21): 11,
  datetime.date(2020, 4, 22): 14,
  datetime.date(2020, 4, 23): 3,
  datetime.date(2020, 4, 24): 7,
  datetime.date(2020, 4, 25): 6,
  datetime.date(2020, 4, 26): 6,
  datetime.date(2020, 4, 27): 0,
  datetime.date(2020, 4, 28): 1,
  datetime.date(2020, 4, 29): 1,
  datetime.date(2020, 4, 30): 2,
  datetime.date(2020, 5, 2): 0,
  datetime.date(2020, 5, 3): 0,
  datetime.date(2020, 5, 1): 0,
  datetime.date(2020, 5, 4): 9,
  datetime.date(2020, 5, 5): 8,
  datetime.date(2020, 5, 6): 9,
  datetime.date(2020, 5, 8): 12,
  datetime.date(2020, 5, 9): 6,
  datetime.date(2020, 5, 7): 10,
  datetime.date(2020, 5, 10): 8,
  datetime.date(2020, 5, 11): 17,
  datetime.date(2020, 5, 12): 18,
  datetime.date(2020, 5, 13): 17,
  datetime.date(2020, 5, 14): 15,
  datetime.date(2020, 5, 15): 16,
  datetime.date(2020, 5, 16): 8,
  datetime.date(2020, 5, 17): 6,
  datetime.date(2020, 5, 18): 17,
  datetime.date(2020, 5, 19): 16,

In [ ]:
# Check total staff constituent by pass type using profile types
for profile in both_sites_df["Prof"].unique():
    number_staff = both_sites_df[both_sites_df["Prof"]==profile]["Card"].nunique()
    print(f"{profile}: {number_staff}")

Visitor Pass: 21
Temp Pass: 947
Staff Pass: 1520


In [ ]:
# Try identifying the card ID for each department basedon staff pass by using daily tap in records

for date in both_sites_df["Date"].unique():
    for dept in both_sites_df["Dept"].unique():
        both_sites_df[(both_sites_df["Dept"] == dept) & (both_sites_df["Date"]== "Date")]
        unique_staff_cards_list = both_sites_df[]
        print("")

In [ ]:
numeric_card_records_df["Card"].nunique()

2443

Question 1.2
Using the prepared data from question 1.1, create dashboard(s) for end-users to self-serve to report and make decisions on the following issues on a regular basis.

a) The company’s target is to have more than 80% of staff working from home at any time.

b) For those staff who are not able to work from home, the company wants to know how many days per week are they coming in and to stagger their work hours and ensure an even proportion are coming in at the three different time slots. The proportion should roughly be 1/3 for each slot.

i. 0700 to 0829

ii. 0830 to 0929

iii. 0930 to 1030

c) For individuals issued with Temp or Visitor Pass, the company wants to know the number at each office site.